In [53]:
import re
import os
try:
    import xml.etree.cElementTree as ET
except ImportError:
    import xml.etree.ElementTree as ET

In [51]:
filelist = ['PART_I.txt', 'PART_II.txt', 'PART_III.txt']

In [55]:
def convert_xml(filename, base=None, converted_file='converted.tmp'):
    # convert file to a valid xml file.
    regex = re.compile(r'(?P<number>\d+)>')
    if base is None:
        fpout = open(converted_file, 'w')
        filepath = filename
    else:
        fpout = open(os.path.join(base, converted_file), 'w')
        filepath = os.path.join(base, filename)
    fpout.write('<root>\n')
    for line in open(filepath):
        txt = regex.sub(lambda x: '"%s">' % x.group('number'), line)
        if '<doc' in txt or '</doc>' in txt or 'summary>' in txt or 'short_text>' in txt:
            pass
        else:
            txt = txt.replace('&', '&amp;').replace('<', '&lt;').replace('>', '&gt;').replace('"', '&quot;').replace("'", '&apos;')
        fpout.write(txt)
    fpout.write('</root>')
    fpout.close()

In [62]:
def convert_data(filename,  base=None, converted_file='converted.tmp'):
    if base is None:
        os.makedirs('converted', exist_ok=True)
        fptrain = open(os.path.join('converted', filename), 'w')
    else:
        os.makedirs(os.path.join(base, 'converted'), exist_ok=True)
        fptrain = open(os.path.join(base, 'converted', filename), 'w')
        converted_file = os.path.join(base, converted_file)
    xml = ET.ElementTree(file=converted_file)
    root = xml.getroot()
    for i, doc in enumerate(root):
        summary = doc.find('summary').text.strip()
        text = doc.find('short_text').text.strip()
        text = '<d><p><s>' + text.replace('。', '。</s><s>') + '</s></p></d>'
        fptrain.write('publisher=HIT\tabstract=<d><p><s>{summary}</s></p></d>\t{text}\n'.format(summary=summary, text=text))
    fptrain.close()

In [63]:
for filename in filelist:
    convert_xml(filename, base='LCSTS/DATA')
    convert_data(filename, base='LCSTS/DATA')